In [ ]:
import numpy as np


In [ ]:
datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

In [ ]:
print(datain)
print(datain.shape)   # Useful if it’s an array
print(type(datain)) 

In [ ]:
print(dataout)
print(dataout.shape)   # Useful if it’s an array
print(type(dataout)) 

# Week 1

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Initial data ---
#X_init = np.array([...])  # your 15x3 input array
#y_init = np.array([...])  # your 15 outputs

X_init = datain
y_init = dataout

In [ ]:
# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_init)

# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu[0]
    sigma = sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative because minimize()

# --- Optimize acquisition function with multiple random starts ---
y_best = y_init.max()
bounds = [(0,1), (0,1), (0,1)]

best_x = None
best_ei = float('inf')
for _ in range(10):
    x0 = np.random.rand(3)
    res = minimize(lambda x: expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
print("Next compound combination to try:", x_next)


#Week 2

In [ ]:
import numpy as np

# Add the new observation
X_new = np.array([[0.901884, 0.273619, 0.723590]])
y_new = np.array([-0.13917950276182944])

# Combine with previous data
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])

X_init = X_all
y_init = y_all

print(X_init)
print(y_init)


In [ ]:
# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_init)

In [ ]:
# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu[0]
    sigma = sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative because minimize()

In [ ]:
# --- Optimize acquisition function with multiple random starts ---
y_best = y_init.max()
bounds = [(0,1), (0,1), (0,1)]

best_x = None
best_ei = float('inf')
for _ in range(30):
    x0 = np.random.rand(3)
    res = minimize(lambda x: expected_improvement(x, gp, y_best, xi=0.01),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
print("Next compound combination to try:", x_next)